In [ ]:
test = merged_df.loc[100]
test
image = getImagePath(test['image_id'])
xyxy = [test['x_1'], test['y_1'], test['width'], test['height']]
xyxy

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

# Replace 'path_to_your_image.jpg' with the actual path to your image file
image_path = getImagePath(test['image_id'])
image = io.imread(image_path)

# Bounding box coordinates and dimensions
bbox = xyxy

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(image)

# Create a Rectangle patch
rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

# Set the x and y axis limits to match the image size if needed
ax.set_xlim(0, image.shape[1])
ax.set_ylim(image.shape[0], 0)  # Y-axis is inverted for images

# Show the plot with the bounding box
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
from ultralytics import YOLO, SAM
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

def load_image(source_image_path):
    image = cv2.imread(source_image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def get_bounding_box(yolo_model, image):
    person_class_index = 0 # ultralytics class dictionary
    results= yolo_model.predict(image, conf=0.5, classes=[person_class_index], verbose=False)
    for result in results:
        if result:
            boxes = result.boxes
            bbox = boxes.xyxy.tolist()[0]  

        else:
            height, width, _ = image.shape
            bbox = [0, 0, width, height]  
    return bbox

def apply_segmentation(sam_model, image, bbox):
    sam_output = sam_model.predict(image, bboxes=bbox, verbose=False)
    mask_object = sam_output[0].masks
    mask_tensor = mask_object.data
    mask_np = mask_tensor.numpy()
    mask_2d = mask_np.squeeze(axis=0)
    return mask_2d

def apply_mask(image, mask_2d):
    masked_image = np.zeros_like(image)
    for c in range(image.shape[2]):  # Assuming image has 3 channels
        masked_image[..., c] = image[..., c] * mask_2d
    return masked_image

def crop_and_resize(masked_image, mask_2d, resize_dims):
    y_indices, x_indices = np.where(mask_2d)
    x_min, x_max = x_indices.min(), x_indices.max()
    y_min, y_max = y_indices.min(), y_indices.max()
    cropped_image = masked_image[y_min:y_max+1, x_min:x_max+1]
    resized_image = cv2.resize(cropped_image, resize_dims)
    normalized_image = resized_image / 255.0  # Scale pixel values to [0, 1]
    return normalized_image

def augment_images(images, augmenter):
    augmented_images = []
    for img in images:
        # Check if image data type is uint8, if not convert to uint8
        if img.dtype != np.uint8:
            img_uint8 = (img * 255).astype(np.uint8)
            augmented = augmenter(image=img_uint8)
        else:
            augmented = augmenter(image=img)
        augmented_images.append(augmented)
    return augmented_images

def process_batch(batch, yolo_model, sam_model, resize_dims, augmenter, pbar=None):
    #batch here is batch_paths
    batch_results = []
    for source_image_path in batch:
        try:
            image = load_image(source_image_path)
            bbox = get_bounding_box(yolo_model, image)
            mask_2d = apply_segmentation(sam_model, image, bbox)
            masked_image = apply_mask(image, mask_2d)
            normalized_image = crop_and_resize(masked_image, mask_2d, resize_dims)
            batch_results.append(normalized_image)
        except Exception as e:
            print(f"Failed to process image {source_image_path}: {e}")
            if pbar is not None:
                pbar.update(1)  # Update the progress bar even if there's an error
            continue  # Skip the rest of the loop and proceed with the next image
        if pbar is not None:
            pbar.update(1) 
    try:
        augmented_batch = augment_images(batch_results, augmenter)
#         display_batch(augmented_batch)
        save_batch_images(batch_results, batch, CELEBA_DATA_PATH)
    except Exception as e:
        print(f"Failed to augment and save images: {e}")
    return augmented_batch

def display_batch(images, figsize=(8, 8), columns=5):
    plt.figure(figsize=figsize)
    for i, image in enumerate(images):
        plt.subplot(len(images) // columns + 1, columns, i + 1)
        plt.imshow(image)
        plt.text(5, 15, str(i), color='white', fontsize=12, weight='bold')
        plt.axis('off')
    plt.tight_layout()
    plt.show()    

def process_partition(df_partition, yolo_model, sam_model, resize_dims, augmenter, batch_size=32):
    num_images = len(df_partition)
    processed_images = []
    with tqdm(total=num_images, desc="Processing partition", unit="img") as pbar:
        for start_idx in range(0, num_images, batch_size):
            end_idx = min(start_idx + batch_size, num_images)
            # get image paths from the partitioned_df
            batch_paths = [getImagePath(df_partition.loc[i]['image_id']) for i in range(start_idx, end_idx)]
            processed_batch = process_batch(batch_paths, yolo_model, sam_model, resize_dims, augmenter, pbar)
            processed_images.extend(processed_batch)
    return processed_images

def save_batch_images(batch_images, batch_image_paths, base_output_dir):
    """
    Save a batch of images to the specified base output directory.

    :param batch_images: List of image data to be saved.
    :param batch_image_paths: List of source paths of images, used to extract the image IDs.
    :param base_output_dir: Base directory where the 'processed_img' folder will be created and images will be saved.
    """
    # Create the output directory if it does not exist
    output_dir = os.path.join(base_output_dir, 'processed_img')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save each image in the batch
    for img, path in zip(batch_images, batch_image_paths):
        image_id = os.path.basename(path)
        output_image_path = os.path.join(output_dir, image_id)
        img_pil = Image.fromarray((img * 255).astype('uint8'))  # Convert from [0,1] to [0,255] and to uint8
        img_pil.save(output_image_path, format="JPEG")


In [ ]:
from ultralytics.utils.torch_utils import select_device

#change to gpu or empty string to auto-select first avail
selected_device = select_device(device='mps', batch=32, newline=False, verbose=True) 
print(f"Using device: {selected_device}")

# Initialize models
yolo_model = YOLO('yolov8s.pt')
sam_model = SAM('mobile_sam.pt')


augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flips
    iaa.Crop(percent=(0, 0.1)),  # Random crops
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True)


# Example usage
resize_dims = (224, 224)
train_processed = process_partition(train_df, yolo_model, sam_model, resize_dims, augmenter)
train_processed = process_partition(val_df, yolo_model, sam_model, resize_dims, augmenter)
train_processed = process_partition(test_df, yolo_model, sam_model, resize_dims, augmenter)

In [ ]:
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import torch

def calculate_weights_to_tensor(merged_path):
    # Read the CSV file
    df = pd.read_csv(merged_path)

    # Get class counts for the 'Male' attribute
    male_class_counts = df['Male'].value_counts().sort_index()

    # Alternatively, compute class weights using sklearn's utility function
    male_class_weights_sklearn = compute_class_weight(
        class_weight='balanced',
        classes=[0, 1],
        y=df['Male'].values
    )
    
    # Store the class weights in a dictionary
    class_weights_dict = {'Male': {0: male_class_weights_sklearn[0], 1: male_class_weights_sklearn[1]}}

    # first two columns are 'img_id', 'male', 'Partition', we skip them
    # The rest are the attributes you want to calculate class weights for
    attribute_columns = df.columns[3:]

    # Iterate over the attribute columns to calculate class weights
    for attribute in attribute_columns:
        # Compute class weights for the current attribute
        class_weights = compute_class_weight(
            class_weight='balanced', 
            classes=[0, 1], 
            y=df[attribute].values
        )
        
        # Store the weights in the dictionary
        class_weights_dict[attribute] = {0: class_weights[0], 1: class_weights[1]}

    # Print the class weights
    for attribute, weights in class_weights_dict.items():
        print(f"Class weights for {attribute}: {weights}\n")
    
    # Convert the class weights dictionary to a tensor
    weights_tensor = torch.tensor([list(weights.values()) for _, weights in class_weights_dict.items()])

    # This tensor will be 2D with shape [num_attributes, 2], where the second dimension contains the weights for classes 0 and 1
    return weights_tensor

# Usage example:
weights_tensor = calculate_weights_to_tensor(merged_path)
print(weights_tensor)
